# Creating a Simple Agent with Tracing

In [1]:
import dotenv
import os

from openai import OpenAI

dotenv.load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    print(
        """Error: OPENAI_API_KEY environment variable not set. Please copy the .env.template file as .env and fill it in.
    
    You can execute these commands in the terminal to get started:
    cp .env.template .env
    code .env
    """
    )

# Test OpenAI Access
print(
    OpenAI()
    .responses.create(
        model=os.environ["OPENAI_DEFAULT_MODEL"], input="Say: We are up and running!"
    )
    .output_text
)

We are up and running!


In [2]:
from agents import Agent, Runner, trace
from openai.types.responses import ResponseTextDeltaEvent

Create a simple Nutrition Assistant Agent

In [ ]:
nutrition_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful assistant giving out nutrition advice.
    You give concise answers.
    """,
)

# The above is Anthropic's Agent SDK/framework but with it we are using the OpenAI API key.
'''
SDK (Software Development Kit) = The framework/library we use to build our agent.
LLM API = The actual language model service we're calling.
They are separate and independent!

The SDK is just the scaffolding. We can plug in whatever LLM API we want (OpenAI, Anthropic, Google, etc.) 
as long as the SDK supports it.
'''
# In the above, the 'instructions' acts as a system prompt.

Let's execute the Agent:

In [ ]:
with trace("Simple Nutrition Agent"):
    result = await Runner.run(nutrition_agent, "How healthy are bananas?")

print(result)

# The 'run' function is an asynchronous function in python.

# The POST request on traces that we see is not the POST request sent to LLM. The POST request is storing new trace data on a tracing platform like LangSmith.

# Also the request we send to LLM to receive answer to our input question is actually a POST request and not a GET request.

# POST requests:

# - Used to send data to create/process something
# - Data is sent in the request body
# - Expected to cause processing/changes

RunResult:
- Last agent: Agent(name="Nutrition Assistant", ...)
- Final output (str):
    Bananas are healthy and convenient:
    
    - Nutrients: good source of potassium, vitamin B6, vitamin C, and dietary fiber.
    - Energy: about 90–110 kcal per medium banana; natural sugars but balanced by fiber.
    - Benefits: supports heart health, digestion, and energy for workouts.
    - Variants: ripe bananas have more sugars; unripe have more resistant starch (gentler on blood sugar).
    - Cautions: generally fine for most people; those with kidney disease may need to limit potassium. Rare allergy.
    
    Overall, a common, nutrient-rich snack or part of meals.
- 2 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


Streaming the answer to the screen, token by token

In [ ]:
response_stream = Runner.run_streamed(nutrition_agent, "How healthy are bananas?")

async for event in response_stream.stream_events():
    if event.type == "raw_response_event" and isinstance(
        event.data, ResponseTextDeltaEvent
    ):
        print(event.data.delta, end="", flush=True)

'''
async for - Like a regular for loop, but for asynchronous operations

- It waits for each event to arrive
- Processes it immediately
- Then waits for the next one
- Think of it like checking your mailbox repeatedly as letters arrive

- response_stream.stream_events() - Returns events one at a time as they happen

- Each "event" is a notification about something that occurred
- Events arrive in real-time as the LLM generates the response
'''

Bananas are a healthy, convenient fruit. Quick snapshot:

- Nutrients: good source of potassium, vitamin C, vitamin B6, and fiber.
- Health benefits: supports heart health, digestion, and steady energy from natural sugars; resistant starch in greener bananas may aid gut health.
- Considerations: moderate sugar content; portion size matters for diabetes or calorie goals; not a protein source.
- Tips: ripe bananas are sweeter; greener ones have more resistant starch.

One medium banana (about 118 g) is a common reference: ~105 calories.

_Good Job!_